# Rain Hyperparameter Initialization

The purpose of this notebook is to examine the hyperparameter associated with rain in the FMDA RNN model. The current architecture is to have an additive effect of rain on fuel moisture in the initiated model.

## Background: Physics Model from Vanderkamp 2014

Vanderkamp (2014) establish the following physics-based model of the affect of rain on fuels. The depth of a stick is modeled, and there is assumed to be radial symmetry and symmetry along the length of the stick. During rain, the outer layer of the stick absorbs the rain. The amount of absorbed rain $P_{abs}$, in units of $kg/s$, is calculated from the diamter of the stick $2r$ and the incident precipitation rate $P_{inc}$ (in units of $\frac{kg}{sm^2}$):

$P_{abs}=\frac{2r}{P_{inc}}$

Additionally, the moisture is limited to a max value $m_{max}$, which is "determined through model calibration". For 10-h fuels, they arrived on $m_{max}=76.8\%$. This value was valibrated at a single study location.

## Translating the Vanderkamp Model

### RNN Model of Rain

The core of the physics-initiated model is an ODE built off of Mandel 2014. In the current version of the RNN (as of Jan 1 2024), the initiated model adds the effect of rain to the ODE moisture model. Call the ODE component of the model $f(m_t)$. So the modeled moisture at time $t=1$ would be the sum of the moisture model calculated on the previous timestep ($f(m_0)$) and a constant effect based on the rainfall. Let the following definitions for the model be:

* $S_{fm}$: scale parameter associated with fuel moisture. Currently (Jan 1 2014), this is calculated as the maximum observed fuel moisture value in the training data. Moving forward, this will be a hyperparameter.
* $S_{rain}$: scale parameter associated with rain. As before, this is calculated as from the training data as the maximum observed rain (units of mm).
* $R_t$: the observed rainfall data at time $t$ (units of mm)
* $K$: the hyperparameter associated with rain. It is the percent increase in fuel moisture per mm of rain per hour, in units of percentage of fuel moisture per hour millimeter $\frac{\%}{h\cdot mm}$ (??? Get Jan to Confirm)

Thus, the modeled moisture at time $t=1$ is:

$$
m_1 = f(m_0)+ K \cdot \frac{S_{fm}}{S_{rain}} \cdot R_1
$$

In this notebook we will attempt to translate the Vanderkamp model of rain into the physics-initiated RNN to get a good value of the parameter $K$. Following that, we could conduct a grid-search on that parameter around this physics-based starting value.

### Setting Parameter K

First, we will be working with 10h fuels, which corresponds to 1/2-inch sticks, per [USDA](https://www.fs.usda.gov/t-d/pubs/htmlpubs/htm05512347/index.htm#:~:text=Dead%20fuels%20are%20divided%20into,to%20absorb%20or%20lose%20moisture.), which is equal to 12.7 mm. So the numerator in the Vanderkamp model would be 12.7. The denominator is the incoming rain $P_{inc}$, in units of $kg s^{-1} m^{-2}$. The physics-initiated RNN formulation will constrain the rainfall to be 1-mm over an hour...

## Setup

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Setup
import reproducibility

# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import tensorflow as tf
import pickle, os

from data_funcs import load_and_fix_data, plot_data
from moisture_rnn import create_rnn_data, train_rnn, rnn_predict

In [ ]:
# Data
# Change directory for data read/write

dict_file='../data/raws_CO_202306.pickle' # input path of FMDA dictionaries
reproducibility_file='../data/reproducibility_dict.pickle'

# read test datasets
test_dict={}
test_dict.update(load_and_fix_data(dict_file))
print(test_dict.keys())

repro_dict={}
repro_dict.update(load_and_fix_data(reproducibility_file))
print(repro_dict.keys())
# Build Case Data
id = "CPTC2_202306010000"
case_data=test_dict[id]
case_data["hours"]=len(case_data['fm'])
case_data["h2"]=int(24*20)

## References

* Vanderkamp 2014
* Mandel 2014
* USDA Stick Diam: https://www.fs.usda.gov/t-d/pubs/htmlpubs/htm05512347/